In [42]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import yfinance as yf
import warnings

warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['symbol'] = sp500['Symbol'].str.replace('.', '-', regex=False)
symbols = sp500['symbol'].unique().tolist()

# 2. Set date range for data download
end_date = pd.Timestamp('2024-06-01')
start_date = end_date - pd.DateOffset(years=2)

sp500['symbol'] = sp500['Symbol'].str.replace('.', '-')

# 4. Flatten multi-index columns: (attribute, ticker) -> ticker_attribute
df.columns = ['{}_{}'.format(ticker, attr) for attr, ticker in df.columns]

# 5. Convert wide to long format
df_long = df.reset_index().melt(id_vars='Date', var_name='ticker_attr', value_name='value')

start_date = pd.to_datetime(end_date) - pd.DateOffset(years=5)

df = yf.download(tickers=symbols_list, start=start_date, end=end_date).stack()

# 8. Rename columns to lowercase and consistent names
df_tidy = df_tidy.rename(columns={
    'Date': 'date',
    'Adj Close': 'adj_close',
    'Close': 'close',
    'High': 'high',
    'Low': 'low',
    'Open': 'open',
    'Volume': 'volume'
})

# 9. Optional: sort by date and ticker
df_tidy = df_tidy.sort_values(['date', 'ticker']).reset_index(drop=True)

df


[*********************100%***********************]  503 of 503 completed

6 Failed downloads:
['SW', 'VLTO', 'SOLV', 'GEV']: YFPricesMissingError('possibly delisted; no price data found  (1d 2018-09-27 00:00:00 -> 2023-09-27) (Yahoo error = "Data doesn\'t exist for startDate = 1538020800, endDate = 1695787200")')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2018-09-27 00:00:00 -> 2023-09-27)')
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


Price              Adj Close       Close        High         Low        Open  \
Date       Ticker                                                              
2018-09-27 A             NaN   67.387962   67.435548   66.721693   67.178564   
           AAPL          NaN   53.398628   53.752326   53.063922   53.130391   
           ABBV          NaN   69.834373   70.390734   69.700847   69.990154   
           ABT           NaN   64.918686   65.060938   64.625303   64.918686   
           ACGL          NaN   28.137087   28.403339   28.089543   28.175122   
...                      ...         ...         ...         ...         ...   
2023-09-26 XYL           NaN   87.701065   89.004043   87.681475   88.543592   
           YUM           NaN  119.860725  120.566296  119.319458  120.083026   
           ZBH           NaN  110.800156  115.381526  110.760745  115.046540   
           ZBRA          NaN  223.960007  226.649994  222.580002  225.970001   
           ZTS           NaN  173.604691  175.155523  173.015777  173.320051   

Price                   Volume  
Date       Ticker               
2018-09-27 A         1581700.0  
           AAPL    120724800.0  
           ABBV      3028600.0  
           ABT       5493900.0  
           ACGL       723100.0  
...                        ...  
2023-09-26 XYL       1322400.0  
           YUM       1500600.0  
           ZBH       3610500.0  
           ZBRA       355400.0  
           ZTS       1463200.0  

[617202 rows x 6 columns]